<a href="https://colab.research.google.com/github/gchaewon/10th-Ewha-Festival-Front/blob/main/multi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import library

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
from scipy.io import wavfile
from collections import defaultdict, Counter
from scipy import signal
import numpy as np
import librosa
import sklearn
import random
from unicodedata import normalize
from keras.layers import Dense
from keras import Model
from keras import Input
from keras.utils import to_categorical
from keras.regularizers import l2
from keras.layers import Dense, TimeDistributed, Dropout, Bidirectional, GRU, BatchNormalization, Activation, LeakyReLU, LSTM, Flatten, RepeatVector, Permute, Multiply, Conv2D, MaxPooling2D

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Load Data

In [ ]:
DATA_TRAIN = '/content/drive/My Drive/졸프/ML/STT/dataset/train'
DATA_TEST = '/content/drive/My Drive/졸프/ML/STT/dataset/test'

# Preprocessing

In [ ]:
trainset = []
testset = []

train_X = []
train_mfccs = []
train_y = []

test_X = []
test_mfccs = []
test_y = []

pad1d = lambda a, i: a[0: i] if a.shape[0] > i else np.hstack((a, np.zeros(i-a.shape[0])))
pad2d = lambda a, i: a[:, 0:i] if a.shape[1] > i else np.hstack((a, np.zeros((a.shape[0], i-a.shape[1]))))

frame_length = 0.025
frame_stride = 0.0010

In [ ]:
from sklearn import preprocessing

for filename in os.listdir(DATA_TRAIN+ "/"):
  filename = normalize('NFC', filename)
  try:
    if '.wav' not in filename:
      continue

    wav, sr = librosa.load(DATA_TRAIN+ "/"+ filename, sr=16000)

    mfcc = librosa.feature.mfcc(y=wav, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    padded_mfcc = pad2d(mfcc, 40)

    if filename[0] == '어':
      trainset.append((padded_mfcc, 0))
    elif filename[0] == '음':
      trainset.append((padded_mfcc, 1))
    elif filename[0] == '그':
      trainset.append((padded_mfcc, 2))
  except Exception as e:
    print(filename, e)
    raise

random.shuffle(trainset)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The stan

In [ ]:
for filename in os.listdir(DATA_TEST + "/"):
  filename = normalize('NFC', filename)
  try:
    if '.wav' not in filename in filename:
      continue

    wav, sr = librosa.load(DATA_TEST+ "/"+ filename, sr=16000)

    mfcc = librosa.feature.mfcc(y=wav, sr=16000, n_mfcc=100, n_fft=400, hop_length=160)
    mfcc = sklearn.preprocessing.scale(mfcc, axis=1)
    padded_mfcc = pad2d(mfcc, 40)

    if filename[0] == '어':
      testset.append((padded_mfcc, 0))
    elif filename[0] == '음':
      testset.append((padded_mfcc, 1))
    elif filename[0] == '그':
      testset.append((padded_mfcc, 2))
  except Exception as e:
    print(filename, e)
    raise

random.shuffle(testset)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probably very close to 0. 
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:261: UserWarning: Numerical issues were encountered when centering the data and might not be solved. Dataset may contain too large values. You may need to prescale your features.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_data.py:280: UserWarning: Numerical issues were encountered when scaling the data and might not be solved. The standard deviation of the data is probab

In [ ]:
train_mfccs = [a for (a,b) in trainset]
train_y = [b for (a,b) in trainset]

test_mfccs = [a for (a,b) in testset]
test_y = [b for (a,b) in testset]

train_mfccs = np.array(train_mfccs)
train_y = to_categorical(np.array(train_y))

test_mfccs = np.array(test_mfccs)
test_y = to_categorical(np.array(test_y))

print('train_mfccs:', train_mfccs.shape)
print('train_y:', train_y.shape)

print('test_mfccs:', test_mfccs.shape)
print('test_y:', test_y.shape)


train_mfccs: (2033, 100, 40)
train_y: (2033, 3)
test_mfccs: (150, 100, 40)
test_y: (150, 3)


# Training

In [ ]:
train_X_ex = np.expand_dims(train_mfccs, -1)
test_X_ex = np.expand_dims(test_mfccs, -1)
print('train X shape:', train_X_ex.shape)
print('test X shape:', test_X_ex.shape)

train X shape: (2033, 100, 40, 1)
test X shape: (150, 100, 40, 1)


In [ ]:
ip = Input(shape=train_X_ex[0].shape)

m = Conv2D(32, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Conv2D(32*2, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Conv2D(32*3, kernel_size=(4,4), activation='relu')(ip)
m = MaxPooling2D(pool_size=(4,4))(m)

m = Flatten()(m)

m = Dense(64, activation='relu', kernel_regularizer=l2(0.001))(m)

m = Dense(32, activation='relu', kernel_regularizer=l2(0.001))(m)

op = Dense(3, activation='softmax')(m)

model = Model(ip, op)

model.summary()

Model: "model_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_12 (InputLayer)       [(None, 100, 40, 1)]      0         
                                                                 
 conv2d_35 (Conv2D)          (None, 97, 37, 96)        1632      
                                                                 
 max_pooling2d_33 (MaxPooli  (None, 24, 9, 96)         0         
 ng2D)                                                           
                                                                 
 flatten_9 (Flatten)         (None, 20736)             0         
                                                                 
 dense_23 (Dense)            (None, 64)                1327168   
                                                                 
 dense_24 (Dense)            (None, 32)                2080      
                                                           

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

history = model.fit(train_X_ex,
                    train_y,
                    epochs=50,
                    batch_size=128,
                    verbose=1
                    )


Epoch 1/50
16/16 [==============================] - 14s 780ms/step - loss: 1.8393 - accuracy: 0.3369
Epoch 2/50
16/16 [==============================] - 14s 861ms/step - loss: 1.2242 - accuracy: 0.4383
Epoch 3/50
16/16 [==============================] - 13s 808ms/step - loss: 1.1404 - accuracy: 0.4998
Epoch 4/50
16/16 [==============================] - 10s 654ms/step - loss: 1.0574 - accuracy: 0.5622
Epoch 5/50
16/16 [==============================] - 13s 807ms/step - loss: 1.0325 - accuracy: 0.5632
Epoch 6/50
16/16 [==============================] - 13s 823ms/step - loss: 0.9378 - accuracy: 0.6153
Epoch 7/50
16/16 [==============================] - 13s 795ms/step - loss: 0.8532 - accuracy: 0.6808
Epoch 8/50
16/16 [==============================] - 10s 607ms/step - loss: 0.7876 - accuracy: 0.7265
Epoch 9/50
16/16 [==============================] - 13s 807ms/step - loss: 0.7452 - accuracy: 0.7452
Epoch 10/50
16/16 [==============================] - 13s 823ms/step - loss: 0.6630 - accura

# Save model

In [ ]:
model.save('/content/drive/My Drive/졸프/ML/STT/model/multi_model.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
